# Test Archon HEB WAGO

This notebook is used to test a MODS2025 WAGO-based Archon HEB.  It does not
test the Archon proper, but can power it on/off

See HEB_WAGO.ipynb for more detailed description of the WAGO HEB.

In [91]:
from time import sleep
from datetime import datetime, date, time, timedelta, UTC

import sys

# Setup the Modbus I/O client as a synchronous TCP client

from pymodbus.client import ModbusTcpClient as mbc

import numpy as np

# HEB host names

hebHosts = {"mods1b":"192.168.139.142",
            "mods1r":"192.168.139.141",
            "mods2b":"192.168.139.242",
            "mods2r":"192.168.139.241",
            "spare1":"192.168.139.41",
            "spare2":"192.168.139.42"}

# functions we need

def ptRTD2C(rawRTD):
    tempRes = 0.1   # module resolution is 0.1C per ADU
    tempMax = 850.0 # maximum temperature for a Pt RTD in deg C
    wrapT = tempRes*((2.0**16)-1) # ADU wrap at <0C to 2^16-1

    temp = tempRes*rawRTD
    if temp > tempMax:
        temp -= wrapT

    return temp

def getRTDs(client,addr,num):
    temp = []
    try:
        rd = client.read_input_registers(addr,count=num)
    except Exception as ex:
        print(f"** Warning: Cannot read WAGO RTD module - {ex}")
        for i in range(num):
            temp.append(-999.99)
        return False,temp

    for i in range(num):
        temp.append(ptRTD2C(float(rd.registers[i]))) # convert to deg C
       
    return True,temp

def getQCs(client,addr):
    numAI = 4
    qcData = []
    try:
        rd = client.read_holding_registers(addr,count=numAI)
    except Exception as ex:
        print(f"** Warning: Cannot read quad cell board - {ex}")
        for i in range(numAI):
            qcData.append(-999.99)
        return False,qcData
    
    qcData = rd.registers
    return True,qcData

def qc2vdc(rawQC):
    posMax = 2**15 - 1
    negMin = 2**16 - 2
    if rawQC > posMax:
        Vout = 10.0*((rawQC-negMin)/posMax)
    else:
        Vout = 10.0*(rawQC/posMax)
    return Vout

def getDOStatus(client,addr,num):
    states = []
    try:
        rd = client.read_coils(addr,count=num)
    except Exception as ex:
        print(f"** Warning: Cannot read WAGO DO module - {ex}")
        for i in range(num):
            states.append(False)
        return False,states
        
    states = rd.bits
    return True,states

def setDO(client,baseAddr,channel,state,delay=0.1):
    doReg = baseAddr+channel-1
    try:
        rq = client.write_coil(doReg,state)
        sleep(delay) # inject deplay in sec 
        return True
    except Exception as ex:
        print(f"** Warning: Cannot set WAGO channel {chan} on DO module - {ex}")
        return False

### WAGO device configuration

Same for all HEBs

Two (2) Pt RTD temperature sensors: 
  * inside box air temperature
  * dewar LN2 reservoir temperature

Two (2) digital outputs connected to power switching relays:
 * Archon controller AC power (normally open)
 * Vacuum ionization gauge power (normally open)

Quad Cell readout board is connected to a 4-channel analog input module, 1 channel per quadrant.

In [117]:
# 4-channel analog input (ADC) module

qcAddr = 0

# 4-channel RTD readout module

rtdAddr = 4
numRTDs = 2
rtdName = ['HEBTEMP','DEWTEMP','CHAN3','CHAN4']

# 8-channel digital output module

numOut = 2
doAddr = 512

# switching truth table

outName = ['Archon','Ion Gauge']
outTrue = ['ON','ON']
outFalse = ['OFF','OFF']

## Select the HEB to test

In [127]:
unitID = "mods2r"

hebHost = hebHosts[unitID]

# Create a modbus TCP client instance

wagoClient = mbc(hebHost)

# connect 

haveConnect = wagoClient.connect()
sleep(1.0) # enough time for the TCP connection to complete

if haveConnect:
    print(f"Connected to {unitID.upper()} HEB WAGO unit on IP {hebHost}")
else:
    print(f"Cannot connect to {unitID.upper()} - check host ID, power, etc. and try again")

Connected to MODS2R HEB WAGO unit on IP 192.168.139.241


## Read HEB status


In [128]:
# read the RTD(s)

print(f"{unitID.upper()} Sensors and Power State:")

status,rtdTemps = getRTDs(wagoClient,rtdAddr,numRTDs)
if status:
    print("\nPt RTDs:")
    for i, temp in enumerate(rtdTemps):
        print(f"  RTD {i+1}: {rtdName[i]}={rtdTemps[i]:.1f} C")
else:
    print("could not read the HEB WAGO")

# read the quad cell board

status,qcData = getQCs(wagoClient,qcAddr)
if status:
    print("\nQuad Cell:")
    for i, temp in enumerate(qcData):
        vOut = qc2vdc(qcData[i])
        print(f"  Q{i+1}: {int(qcData[i]):5d} adu = {vOut:7.4f} VDC")
else:
    print("could not read the HEB WAGO")

# read the power status

okRead,doState = getDOStatus(wagoClient,doAddr,numOut)
if okRead:
    print("\nPower Status:")
    for i in range(numOut):
        print(f"  {outName[i]}: {(doState[i] and outTrue[i] or outFalse[i])}")
else:
    print("Could not read the WAGO")

wagoClient.close()

MODS2R Sensors and Power State:

Pt RTDs:
  RTD 1: HEBTEMP=25.3 C
  RTD 2: DEWTEMP=20.0 C

Quad Cell:
  Q1:   237 adu =  0.0723 VDC
  Q2:   239 adu =  0.0729 VDC
  Q3:   242 adu =  0.0739 VDC
  Q4:   241 adu =  0.0735 VDC

Power Status:
  Archon: OFF
  Ion Gauge: OFF


## Power-up Test

Turn on the Archon and Ion Gauage

In [129]:
wagoClient = mbc(hebHost)
haveConnect = wagoClient.connect()
sleep(0.5)

if haveConnect:
    print(f"Powering on the {unitID.upper()} Archon and Ion Gauge:")
    for chan in [1,2]:
        if setDO(wagoClient,doAddr,chan,True):
            print(f"  Switching {outName[chan-1]} On")
        else:
            print(f"  **ERROR: Could not switch {outName[chan-1]} On")
    
    sleep(0.5)

    # read the new status

    print("\nStatus:")
    okRead,doState = getDOStatus(wagoClient,doAddr,numOut)
    if okRead:
        for i in range(numOut):
            print(f"  {outName[i]}: {(doState[i] and outTrue[i] or outFalse[i])}")
    else:
        print("Could not read the WAGO")
    
else:
    print(f"**ERROR: Cannot connect to WAGO on {hebHost}")
    
# always close the connection!

wagoClient.close()

Powering on the MODS2R Archon and Ion Gauge:
  Switching Archon On
  Switching Ion Gauge On

Status:
  Archon: ON
  Ion Gauge: ON


## Turn off everything ("All Off")


In [130]:
# Create a modbus TCP client instance

wagoClient = mbc(hebHost)

# connect 

haveConnect = wagoClient.connect()
sleep(0.5)

if haveConnect:
    print(f"Powering {unitID.upper()} Archon and Ion Gauge Off")
    
    # read the current status
    
    okRead,doState = getDOStatus(wagoClient,doAddr,numOut)
    if okRead:
        print("\nCurrent Register Status:")
        for i in range(numOut):
            print(f"  {outName[i]}: {(doState[i] and outTrue[i] or outFalse[i])}")
    else:
        print("Could not read the WAGO")
        
    # sleep for 0.5 second

    sleep(0.5)
    
    # set all channels True
    
    print("Turning outputs Off:")
    for chan in [1,2]:
        offState = False
        if outTrue[chan-1] == 'OFF':
            print('setting offState TRUE')
            offState = True
        else:
            print('setting offState FALSE')
            
        if setDO(wagoClient,doAddr,chan,offState):
            print(f"  Turned {outName[chan-1]} OFF")
        else:
            print(f"  Could not turn {outName[chan-1]} OFF")
    
    sleep(0.5)

    # read the new status
    
    okRead,doState = getDOStatus(wagoClient,doAddr,numOut)
    if okRead:
        print("\nNew Status:")
        for i in range(numOut):
            print(f"  {outName[i]}: {(doState[i] and outTrue[i] or outFalse[i])}")
    else:
        print("Could not read the WAGO")
    
else:
    print(f"**ERROR: Cannot connect to WAGO on {hebHost}")
    
# always close the connection!

wagoClient.close()

Powering MODS2R Archon and Ion Gauge Off

Current Register Status:
  Archon: ON
  Ion Gauge: ON
Turning outputs Off:
setting offState FALSE
  Turned Archon OFF
setting offState FALSE
  Turned Ion Gauge OFF

New Status:
  Archon: OFF
  Ion Gauge: OFF


## Readout the quad cell

runs up to `maxReads` reads of the QC board every `readDelay` seconds.  

In [56]:
import sys

# Create a modbus TCP client instance

wagoClient = mbc(hebHost)

# maximum number of reads

maxReads = 50

# time delay between reads

readDelay = 0.5 # seconds

# volts per ADU

vPerADU = 10.0/(2**15 - 1)

# arrays to hold data

qcList = ['QC1','QC2','QC3','QC4']
qcVout = {}
for qc in qcList:
    qcVout[qc] = []
    
# connect 

haveConnect = wagoClient.connect()
sleep(1.0) # enough time for the TCP connection to complete

if haveConnect:
    print(f"Reading quad cell board {maxReads} times:")
    
    for j in range(maxReads):
        status,qcData = getQCs(wagoClient,qcAddr)
        if status:
            outStr = f'[{j:4d}]'
            for i in range(len(qcData)):
                vOut = qc2vdc(qcData[i])
                outStr += f' {vOut:8.5f}'
                qcVout[qcList[i]].append(vOut)
            sys.stdout.write(f"{outStr}\r")
            sleep(readDelay)
    else:
        print("\nDone, quick stats:")
        # Finishing statistics

        for qc in qcList:
            qcMean = np.mean(qcVout[qc])
            qcStd = np.std(qcVout[qc])
            qcMed = np.median(qcVout[qc])
            print(f'{qc}: mean={qcMean:.5f} median={qcMed:.5f} std={qcStd:.5f}')
        print(f'1 adu = {vPerADU:.5f} VDC')
            
else:
    print(f'\nERROR: Could not connect to HEB WAGO on IP {hebHost}')

# always close the connection!

wagoClient.close()

Reading quad cell board 50 times:
[  49]  0.08637  0.08606  0.08515  0.08637
Done, quick stats:
QC1: mean=0.08637 median=0.08637 std=0.00000
QC2: mean=0.08595 median=0.08606 std=0.00015
QC3: mean=0.08513 median=0.08515 std=0.00007
QC4: mean=0.08637 median=0.08637 std=0.00000
1 adu = 0.00031 VDC
